In [1]:
#| default_exp 00_cachew-for-wikiseealsotitles

In [2]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
#| export
import os,torch,json, torch.multiprocessing as mp, joblib, numpy as np, scipy.sparse as sp

from xcai.main import *
from xcai.basics import *

from xcai.models.cachew import CAW001, CachewConfig

In [5]:
os.environ['WANDB_MODE'] = 'disabled'

In [6]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
os.environ['WANDB_PROJECT'] = 'mogicX_01-wikiseealsotitles'

## Setup

In [15]:
output_dir = '/scratch/scai/phd/aiz218323/outputs/mogicX/00_cachew-for-wikiseealsotitles'

data_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/'
config_file = 'wikiseealsotitles'
config_key = 'data_meta'

meta_embed_init_file = '/data/OGB_Weights/LF-WikiSeeAlsoTitles-320K/emb_weights.npy'

meta_name = 'cat'

In [16]:
do_train_inference = False
do_test_inference = False

save_train_inference = False
save_test_inference = False

save_representation = False

use_sxc_sampler, only_test, use_pretrained = True, False, False

pkl_dir = '/scratch/scai/phd/aiz218323/datasets/processed/'

In [17]:
pkl_file = f'{pkl_dir}/cachew/wikiseealsotitles_data-meta_distilbert-base-uncased'
pkl_file = f'{pkl_file}_sxc' if use_sxc_sampler else f'{pkl_file}_xcs'
if only_test: pkl_file = f'{pkl_file}_only-test'
pkl_file = f'{pkl_file}.joblib'

In [18]:
do_inference = do_train_inference or do_test_inference or save_train_inference or save_test_inference or save_representation

In [20]:
os.makedirs(os.path.dirname(pkl_file), exist_ok=True)
block = build_block(pkl_file, config_file, use_sxc_sampler, config_key, n_slbl_samples=4, main_oversample=False, n_sdata_meta_samples=5, 
                    meta_oversample=False, train_meta_topk=5, test_meta_topk=3, data_dir=data_dir)

/home/scai/phd/aiz218323/.local/lib/python3.10/site-packages/xclib-0.97-py3.10-linux-x86_64.egg/xclib/data/data_utils.py:263: UserWarning: Header mis-match from inferred shape!
  warnings.warn("Header mis-match from inferred shape!")


In [21]:
args = XCLearningArguments(
    output_dir=output_dir,
    logging_first_step=True,
    per_device_train_batch_size=512,
    per_device_eval_batch_size=512,
    representation_num_beams=200,
    representation_accumulation_steps=10,
    save_strategy="steps",
    eval_strategy="steps",
    eval_steps=5000,
    save_steps=5000,
    save_total_limit=5,
    num_train_epochs=300,
    predict_with_representation=True,
    adam_epsilon=1e-6,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=2e-4,
    representation_search_type='BRUTEFORCE',

    output_representation_attribute='data_enriched_repr',
    label_representation_attribute='data_repr',
    metadata_representation_attribute='data_repr',
    data_augmentation_attribute='data_repr',
    representation_attribute='data_enriched_repr',
    clustering_representation_attribute='data_enriched_repr',

    group_by_cluster=True,
    num_clustering_warmup_epochs=10,
    num_cluster_update_epochs=5,
    num_cluster_size_update_epochs=25,
    use_data_metadata_for_clustering=True,
    clustering_type='EXPO',
    minimum_cluster_size=2,
    maximum_cluster_size=1600,

    metric_for_best_model='P@1',
    load_best_model_at_end=True,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',

    use_distributional_representation=False,
    use_encoder_parallel=True,
    max_grad_norm=None,
    fp16=True,

    label_names=['lbl2data_idx', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'lbl2data_data2ptr', 'plbl2data_idx', 'plbl2data_data2ptr',
                 f'{meta_name}2data_idx', f'{meta_name}2data_input_ids', f'{meta_name}2data_attention_mask', f'{meta_name}2data_data2ptr'],
                 
    prune_metadata=False,
    num_metadata_prune_warmup_epochs=10,
    num_metadata_prune_epochs=5,
    metadata_prune_batch_size=2048,
    prune_metadata_names=[f'{meta_name}_meta'],
    use_data_metadata_for_pruning=True,

    predict_with_augmentation=False,
    use_augmentation_index_representation=True,

    data_aug_meta_name=meta_name,
    augmentation_num_beams=None,
    data_aug_prefix=meta_name,
    use_label_metadata=False,

    data_meta_batch_size=2048,
    augment_metadata=False,
    num_metadata_augment_warmup_epochs=10,
    num_metadata_augment_epochs=5,

    use_cpu_for_searching=True,
    use_cpu_for_clustering=True,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

config = CachewConfig(
    top_k_metadata = 5,
    num_metadata=block.train.dset.meta['cat_meta'].n_meta,

    data_aug_meta_prefix='cat2data', 
    lbl2data_aug_meta_prefix=None,

    batch_size=bsz, 
    num_batch_labels=5000, 
    margin=0.3,
    num_negatives=10,
    tau=0.1,
    apply_softmax=True,

    calib_margin=0.3,
    calib_num_negatives=10,
    calib_tau=0.1,
    calib_apply_softmax=False,
    calib_loss_weight=0.1,
    use_calib_loss=True,

    use_query_loss=True, 
    use_encoder_parallel=True
)

model = CAW001.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', config=config)
model.init_heads_to_identity()

meta_embeddings = torch.tensor(np.load(meta_embed_init_file), dtype=torch.float32)
model.set_memory_embeddings(meta_embeddings)

In [22]:
metric = PrecReclMrr(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                     pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200], mk=[5, 10, 20])

In [ ]:
learn = XCLearner(
    model=model,
    args=args,
    train_dataset=block.train.dset,
    eval_dataset=block.test.dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

main(learn, input_args, n_lbl=block.n_lbl)

## Driver

In [ ]:
#| export
if __name__ == '__main__':
    output_dir = '/scratch/scai/phd/aiz218323/outputs/mogicX/00_cachew-for-wikiseealsotitles'

    data_dir = '/home/scai/phd/aiz218323/scratch/datasets/benchmarks/'
    config_file = 'wikiseealsotitles'
    config_key = 'data_meta'
    
    meta_embed_init_file = '/data/OGB_Weights/LF-WikiSeeAlsoTitles-320K/emb_weights.npy'
    
    meta_name = 'cat'

    input_args = parse_args()

    pkl_file = f'{input_args.pickle_dir}/cachew/wikiseealsotitles_data-meta_distilbert-base-uncased'
    pkl_file = f'{pkl_file}_sxc' if input_args.use_sxc_sampler else f'{pkl_file}_xcs'
    if input_args.only_test: pkl_file = f'{pkl_file}_only-test'
    pkl_file = f'{pkl_file}.joblib'

    do_inference = input_args.do_train_inference or input_args.do_test_inference or input_args.save_train_prediction or input_args.save_test_prediction or input_args.save_representation

    os.makedirs(os.path.dirname(pkl_file), exist_ok=True)
    block = build_block(pkl_file, config_file, input_args.use_sxc_sampler, config_key, do_build=input_args.build_block, only_test=input_args.only_test, 
                        n_slbl_samples=4, main_oversample=False, n_sdata_meta_samples=5, meta_oversample=False, train_meta_topk=5, test_meta_topk=3)

    args = XCLearningArguments(
        output_dir=output_dir,
        logging_first_step=True,
        per_device_train_batch_size=512,
        per_device_eval_batch_size=512,
        representation_num_beams=200,
        representation_accumulation_steps=10,
        save_strategy="steps",
        eval_strategy="steps",
        eval_steps=5000,
        save_steps=5000,
        save_total_limit=5,
        num_train_epochs=300,
        predict_with_representation=True,
        adam_epsilon=1e-6,
        warmup_steps=100,
        weight_decay=0.01,
        learning_rate=2e-4,
        representation_search_type='BRUTEFORCE',
    
        output_representation_attribute='data_enriched_repr',
        label_representation_attribute='data_repr',
        metadata_representation_attribute='data_repr',
        data_augmentation_attribute='data_repr',
        representation_attribute='data_enriched_repr',
        clustering_representation_attribute='data_enriched_repr',
    
        group_by_cluster=True,
        num_clustering_warmup_epochs=10,
        num_cluster_update_epochs=5,
        num_cluster_size_update_epochs=25,
        use_data_metadata_for_clustering=True,
        clustering_type='EXPO',
        minimum_cluster_size=2,
        maximum_cluster_size=1600,

        metric_for_best_model='P@1',
        load_best_model_at_end=True,
        target_indices_key='plbl2data_idx',
        target_pointer_key='plbl2data_data2ptr',
    
        use_distributional_representation=False,
        use_encoder_parallel=True,
        max_grad_norm=None,
        fp16=True,
    
        label_names=[f'{meta_name}2data_idx', f'{meta_name}2data_input_ids', f'{meta_name}2data_attention_mask', f'{meta_name}2data_data2ptr'],
                     
        prune_metadata=False,
        num_metadata_prune_warmup_epochs=10,
        num_metadata_prune_epochs=5,
        metadata_prune_batch_size=2048,
        prune_metadata_names=[f'{meta_name}_meta'],
        use_data_metadata_for_pruning=True,
    
        predict_with_augmentation=False,
        use_augmentation_index_representation=True,
    
        data_aug_meta_name=meta_name,
        augmentation_num_beams=None,
        data_aug_prefix=meta_name,
        use_label_metadata=False,
    
        data_meta_batch_size=2048,
        augment_metadata=False,
        num_metadata_augment_warmup_epochs=10,
        num_metadata_augment_epochs=5,
    
        use_cpu_for_searching=True,
        use_cpu_for_clustering=True,
    )

    bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

    config = CachewConfig(
        top_k_metadata = 5,
        num_metadata=block.train.dset.meta['cat_meta'].n_meta,
    
        data_aug_meta_prefix='cat2data', 
        lbl2data_aug_meta_prefix=None,
    
        batch_size=bsz, 
        num_batch_labels=5000, 
        margin=0.3,
        num_negatives=10,
        tau=0.1,
        apply_softmax=True,
    
        calib_margin=0.3,
        calib_num_negatives=10,
        calib_tau=0.1,
        calib_apply_softmax=False,
        calib_loss_weight=0.1,
        use_calib_loss=True,
    
        use_query_loss=True, 
        use_encoder_parallel=True
    )
    
    model = CAW001.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', config=config)
    model.init_heads_to_identity()
    
    meta_embeddings = torch.tensor(np.load(meta_embed_init_file), dtype=torch.float32)
    model.set_memory_embeddings(meta_embeddings)

    metric = PrecReclMrr(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                         pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200], mk=[5, 10, 20])

    learn = XCLearner(
        model=model,
        args=args,
        train_dataset=block.train.dset,
        eval_dataset=block.test.dset,
        data_collator=block.collator,
        compute_metrics=metric,
    )
    
    main(learn, input_args, n_lbl=block.n_lbl)
    